# 🎯 Apa Itu Masked Attention?
- Masked attention adalah mekanisme untuk mencegah model melihat ke depan (future tokens) saat memproses urutan input.
- Biasanya dilakukan dengan masking bagian atas segitiga dari matriks attention (upper triangular matrix), sehingga:
    - Saat memproses token ke-i, model hanya bisa "melihat" token ke-0 sampai ke-i.
    - Token berikutnya disembunyikan (di-masking) dengan -∞ sebelum softmax.

## Mengapa Perlu Masked Attention?
✅ 1. Agar konsisten dengan cara kita generate teks
Transformer decoder digunakan untuk generate satu token demi satu.

✅ 2. Autoregressive consistency
Masked attention membuat proses training dan inference konsisten:

- Di training, model belajar memprediksi token berikutnya berdasarkan konteks sebelumnya.
- Di inference, model memang hanya tahu token sebelumnya.
- Tanpa masking, model belajar dari informasi yang tidak akan tersedia saat prediksi nyata dilakukan.


## Jika tidak memakai masked self-attention
Jika model bisa melihat token ke depan (misalnya dalam konteks self-attention di Transformer tanpa menggunakan masked attention), maka model akan melanggar prinsip kausalitas, yang berarti model akan memiliki informasi yang tidak seharusnya tersedia saat melakukan prediksi. Ini akan berdampak buruk pada kualitas prediksi dan konsistensi selama training maupun inference.

Berikut adalah akibat utama jika model bisa melihat token ke depan, beserta contoh nyata:

1. Prediksi Tidak Realistis (Cheating)
Model akan belajar "menipu" dirinya sendiri dengan menggunakan informasi dari masa depan untuk memprediksi token saat ini. Ini tidak realistis dalam konteks banyak aplikasi, seperti generasi teks atau penerjemahan bahasa.

2. Overfitting pada Data Training
Model yang bisa melihat ke depan akan cenderung terlalu mengandalkan informasi masa depan yang ada di data pelatihan, sehingga tidak dapat generalisasi dengan baik pada data yang tidak terlihat (data testing). Model bisa menjadi terlalu bergantung pada pola yang tidak ada di dunia nyata (karena dalam real-world inference, kita tidak bisa melihat masa depan).

3. Kesalahan dalam Prediksi Urutan Teks
Banyak aplikasi yang membutuhkan urutan token yang tepat (misalnya, generasi teks atau prediksi kata berikutnya). Jika model dapat melihat ke depan, maka model akan menghasilkan urutan teks yang tidak masuk akal atau tidak koheren.

4. Model Tidak Dapat Beroperasi dengan Keterbatasan Seperti Autoregressive Generation

In [4]:
import torch ## torch let's us create tensors and also provides helper functions
import torch.nn as nn ## torch.nn gives us nn.module() and nn.Linear()
import torch.nn.functional as F # This gives us the softmax()

## Create Function to Calculate Self-Attention Score
$$ Attention(Q,K,V,M) = Softmax(QK^T/\sqrt{d_k} + M)*V $$

In [6]:
class MaskedSelfAttention(nn.Module): 
                            
    def __init__(self, d_model=2,  
                 row_dim=0, 
                 col_dim=1):
        
        super().__init__()
        
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        
        self.row_dim = row_dim
        self.col_dim = col_dim

        
    def forward(self, token_encodings, mask=None):

        q = self.W_q(token_encodings)
        k = self.W_k(token_encodings)
        v = self.W_v(token_encodings)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            ## Here we are masking out things we don't want to pay attention to
            ##
            ## We replace values we wanted masked out
            ## with a very small negative number so that the SoftMax() function
            ## will give all masked elements an output value (or "probability") of 0.
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9) # I've also seen -1e20 and -9e15 used in masking

        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)

        attention_scores = torch.matmul(attention_percents, v)

        return attention_scores

## Implementation
Say it we want to generate a poem with command

"Write a poem"

In [8]:
## using word embedding, we get
write = [1.16, 0.23]
a = [0.57, 1.36]
poem = [4.41, -2.16]

In [9]:
## create a matrix of token encodings...
encodings_matrix = torch.tensor([[1.16, 0.23],
                                 [0.57, 1.36],
                                 [4.41, -2.16]])

## set the seed for the random number generator
torch.manual_seed(42)

## create a masked self-attention object
maskedSelfAttention = MaskedSelfAttention(d_model=2,
                               row_dim=0,
                               col_dim=1)

## create the mask so that we don't use
## tokens that come after a token of interest
mask = torch.tril(torch.ones(3, 3))
mask = mask == 0
mask # print out the mask

tensor([[False,  True,  True],
        [False, False,  True],
        [False, False, False]])

## Calculate Q,K,V,M and Masked Self-Attention Score

In [11]:
## calculate masked self-attention
maskedSelfAttention(encodings_matrix, mask)

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)

In [12]:
## print out the weight matrix that creates the queries
maskedSelfAttention.W_q.weight.transpose(0, 1)

tensor([[ 0.5406, -0.1657],
        [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>)

In [13]:
## print out the weight matrix that creates the queries
maskedSelfAttention.W_q.weight.transpose(0, 1)

tensor([[ 0.5406, -0.1657],
        [ 0.5869,  0.6496]], grad_fn=<TransposeBackward0>)

In [14]:
## print out the weight matrix that creates the keys
maskedSelfAttention.W_k.weight.transpose(0, 1)

tensor([[-0.1549, -0.3443],
        [ 0.1427,  0.4153]], grad_fn=<TransposeBackward0>)

In [15]:
## print out the weight matrix that creates the values
maskedSelfAttention.W_v.weight.transpose(0, 1)

tensor([[ 0.6233,  0.6146],
        [-0.5188,  0.1323]], grad_fn=<TransposeBackward0>)

In [16]:
## calculate the queries
maskedSelfAttention.W_q(encodings_matrix)

tensor([[ 0.7621, -0.0428],
        [ 1.1063,  0.7890],
        [ 1.1164, -2.1336]], grad_fn=<MmBackward0>)

In [17]:
## calculate the keys
maskedSelfAttention.W_k(encodings_matrix)

tensor([[-0.1469, -0.3038],
        [ 0.1057,  0.3685],
        [-0.9914, -2.4152]], grad_fn=<MmBackward0>)

In [18]:
## calculate the values
maskedSelfAttention.W_v(encodings_matrix)

tensor([[ 0.6038,  0.7434],
        [-0.3502,  0.5303],
        [ 3.8695,  2.4246]], grad_fn=<MmBackward0>)

In [19]:
q = maskedSelfAttention.W_q(encodings_matrix)
q

tensor([[ 0.7621, -0.0428],
        [ 1.1063,  0.7890],
        [ 1.1164, -2.1336]], grad_fn=<MmBackward0>)

In [20]:
k = maskedSelfAttention.W_k(encodings_matrix)
k

tensor([[-0.1469, -0.3038],
        [ 0.1057,  0.3685],
        [-0.9914, -2.4152]], grad_fn=<MmBackward0>)

In [21]:
sims = torch.matmul(q, k.transpose(dim0=0, dim1=1))
sims

tensor([[-0.0990,  0.0648, -0.6523],
        [-0.4022,  0.4078, -3.0024],
        [ 0.4842, -0.6683,  4.0461]], grad_fn=<MmBackward0>)

In [22]:
scaled_sims = sims / (torch.tensor(2)**0.5)

In [23]:
scaled_sims

tensor([[-0.0700,  0.0458, -0.4612],
        [-0.2844,  0.2883, -2.1230],
        [ 0.3424, -0.4725,  2.8610]], grad_fn=<DivBackward0>)

In [24]:
masked_scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
masked_scaled_sims

tensor([[-6.9975e-02, -1.0000e+09, -1.0000e+09],
        [-2.8442e-01,  2.8833e-01, -1.0000e+09],
        [ 3.4241e-01, -4.7253e-01,  2.8610e+00]],
       grad_fn=<MaskedFillBackward0>)

In [25]:
attention_percents = F.softmax(masked_scaled_sims, dim=1)
attention_percents

tensor([[1.0000, 0.0000, 0.0000],
        [0.3606, 0.6394, 0.0000],
        [0.0722, 0.0320, 0.8959]], grad_fn=<SoftmaxBackward0>)

In [26]:
torch.matmul(attention_percents, maskedSelfAttention.W_v(encodings_matrix))

tensor([[ 0.6038,  0.7434],
        [-0.0062,  0.6072],
        [ 3.4989,  2.2427]], grad_fn=<MmBackward0>)